# Análisis Exploratorio de Datos (EDA)

### Carga de Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Carga de Data

In [2]:
InventarioFinal2016 = pd.read_csv(r"C:\Users\USER\Documents\Henry\Proyecto Final\Datasets\EndInvFINAL12312016.csv")

### Revisión Estructural

In [3]:
InventarioFinal2016.head()

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2016-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2016-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2016-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2016-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2016-12-31


In [4]:
# Verificamos el tamaño original del DataFrame

InventarioFinal2016.shape

(224489, 9)

Se cuenta originalmente con 224,489 filas y 9 columnas.

In [5]:
# Información general de los campos

print(InventarioFinal2016.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224489 entries, 0 to 224488
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InventoryId  224489 non-null  object 
 1   Store        224489 non-null  int64  
 2   City         223205 non-null  object 
 3   Brand        224489 non-null  int64  
 4   Description  224489 non-null  object 
 5   Size         224489 non-null  object 
 6   onHand       224489 non-null  int64  
 7   Price        224489 non-null  float64
 8   endDate      224489 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 15.4+ MB
None


En este análisis se puede apreciar que el campo 'endDate' tiene el tipo de dato object, lo cual no es correcto

In [6]:
# Estadísticas Descriptivas - Variables Cuantitativas

InventarioFinal2016.describe().T

,count,mean,std,min,25%,50%,75%,max
Store,224489.0,43.505740,23.326415,1.00,23.00,44.00,66.00,81.0
Brand,224489.0,14356.370513,13118.467851,58.00,3798.00,8259.00,23965.00,90631.0
onHand,224489.0,21.763988,37.233576,0.00,7.00,12.00,22.00,3676.0
Price,224489.0,23.585583,79.202775,0.49,9.99,14.99,23.49,13999.9


### Tratamiento de Duplicados

In [7]:
# Verificamos filas duplicadas

print(f"Cantidad de filas duplicadas: {InventarioFinal2016.duplicated().sum()}")
print('------------------------')

Cantidad de filas duplicadas: 0
------------------------


Se determinó que no existen duplicados en este DataFrame

### Nulos

In [8]:
# Verificamos la cantidad de valores nulos por campo

print("Cantidad de valores nulos por columna:\n", InventarioFinal2016.isnull().sum(), "\n")
print('------------------------')

Cantidad de valores nulos por columna:
 InventoryId       0
Store             0
City           1284
Brand             0
Description       0
Size              0
onHand            0
Price             0
endDate           0
dtype: int64 

------------------------


Se determinó que existen 1,284 valores nulos en en campo 'City'

# Limpieza y Procesamiento de Datos (ETL)

### Conversión de tipo de dato

In [9]:
# Convertir la columna 'endDate' a datetime

InventarioFinal2016['endDate'] = pd.to_datetime(InventarioFinal2016['endDate'])

In [10]:
# Verificamos que se haya cambiado el tipo de dato 

print (InventarioFinal2016.dtypes)

InventoryId            object
Store                   int64
City                   object
Brand                   int64
Description            object
Size                   object
onHand                  int64
Price                 float64
endDate        datetime64[ns]
dtype: object


Se comprueba de que el tipo de dato se ha modificado a datetime

### Limpieza de columnas

Se detectaron varias irregularidades en el campo 'Size', como diferentes formatos para representar tamaños de productos ("750mL", "375mL 2 Pk", "750mL + 3/") y caracteres no deseados ('/'). Además, se encontraron términos como "2 Pk", "3 Pk" que indicaban productos empaquetados en múltiples unidades

In [11]:
import re

# Se desarrolló una función en Python (count_units_optimized) para extraer y contar las unidades en los "packs" ("2 Pk", "3 Pk")
# Esta función creó una nueva columna ('QuantityPerPack') para reflejar el número de unidades por pack en cada registro
# Función optimizada para contar las unidades

def count_units_optimized(size):
    try:
        # Busca patrones de packs ("Pk") o "+ X/"
        if 'Pk' in size:
            # Extrae el número antes de "Pk"
            match = re.search(r'(\d+)\s*Pk', size)
            if match:
                return int(match.group(1))
        elif '+' in size:
            # Extrae el número después del signo "+"
            match = re.search(r'\+\s*(\d+)', size)
            if match:
                return int(match.group(1))
        # Devuelve 1 si no se encuentra un patrón relevante
        return 1
    except Exception as e:
        # Manejo de errores en caso de entradas no esperadas
        print(f"Error procesando el valor {size}: {e}")
        return 1

# Aplicar la función de forma vectorizada
InventarioFinal2016['QuantityPerPack'] = InventarioFinal2016['Size'].apply(count_units_optimized)

# Limpieza de la columna 'Size'
InventarioFinal2016['Size'] = InventarioFinal2016['Size'].str.replace(r'(\d+\s*Pk)', '', regex=True)
InventarioFinal2016['Size'] = InventarioFinal2016['Size'].str.replace(r'(\+\s*\d+)', '', regex=True)
InventarioFinal2016['Size'] = InventarioFinal2016['Size'].str.replace(r'/', '', regex=True)  # Eliminar el carácter '/'

# Eliminar espacios en blanco adicionales
InventarioFinal2016['Size'] = InventarioFinal2016['Size'].str.strip()

# Mostrar los primeros resultados del DataFrame para validación
print(InventarioFinal2016.head())

         InventoryId  Store          City  Brand                  Description  \
0  1_HARDERSFIELD_58      1  HARDERSFIELD     58  Gekkeikan Black & Gold Sake   
1  1_HARDERSFIELD_62      1  HARDERSFIELD     62     Herradura Silver Tequila   
2  1_HARDERSFIELD_63      1  HARDERSFIELD     63   Herradura Reposado Tequila   
3  1_HARDERSFIELD_72      1  HARDERSFIELD     72         No. 3 London Dry Gin   
4  1_HARDERSFIELD_75      1  HARDERSFIELD     75    Three Olives Tomato Vodka   

    Size  onHand  Price    endDate  QuantityPerPack  
0  750mL      11  12.99 2016-12-31                1  
1  750mL       7  36.99 2016-12-31                1  
2  750mL       7  38.99 2016-12-31                1  
3  750mL       4  34.99 2016-12-31                1  
4  750mL       7  14.99 2016-12-31                1  


In [12]:
# Nuevamente verificamos los tipos de datos 

print (InventarioFinal2016.dtypes)

InventoryId                object
Store                       int64
City                       object
Brand                       int64
Description                object
Size                       object
onHand                      int64
Price                     float64
endDate            datetime64[ns]
QuantityPerPack             int64
dtype: object


### Completar los valores nulos

In [13]:
# Se sabe que estos nulos pertenecen a la misma tienda
# Reemplazar valores nulos en la columna 'City' con un valor específico, como 'UNKNOWN'

InventarioFinal2016['City'].fillna('UNKNOWN', inplace = True)

C:\Users\USER\AppData\Local\Temp\ipykernel_24628\1454200678.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  InventarioFinal2016['City'].fillna('UNKNOWN', inplace = True)


### Verificar los valores nulos

In [14]:
# Verificar si aún quedan valores nulos

print("Valores nulos en 'City' después de reemplazo:", InventarioFinal2016['City'].isnull().sum())

Valores nulos en 'City' después de reemplazo: 0


In [15]:
# Nuevamente verificamos la cantidad de valores nulos por campo

print("Cantidad de valores nulos por columna:\n", InventarioFinal2016.isnull().sum(), "\n")
print('------------------------')

Cantidad de valores nulos por columna:
 InventoryId        0
Store              0
City               0
Brand              0
Description        0
Size               0
onHand             0
Price              0
endDate            0
QuantityPerPack    0
dtype: int64 

------------------------


Se puede visualizar que ya no contamos con valores nulos

### Renombrar los campos

In [16]:
# Para un mejor entendimiento, se han renombrado algunos campos

InventarioFinal2016 = InventarioFinal2016.rename(columns={
    'InventoryID': 'InventoryFinalID',
    'Store': 'StoreID',
    'Brand': 'BrandID',
    'onHand': 'OnHand',
    'endDate': 'EndDate'
    })

In [17]:
# Verificar que se hayan modificado correctamente los datos 

print(InventarioFinal2016.dtypes)

InventoryId                object
StoreID                     int64
City                       object
BrandID                     int64
Description                object
Size                       object
OnHand                      int64
Price                     float64
EndDate            datetime64[ns]
QuantityPerPack             int64
dtype: object


### Guardar la data en archivo .CSV

In [18]:
InventarioFinal2016.to_csv('InventarioFinal2016.csv', index = False)